In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [15]:
'''
Set local I/O
'''

# Set folder and filename for rail GIS file
rail_folder = '../data/railways/LAMR/merged/'
rail_file = rail_folder+'LAMR.shp'

# Set folder and filename for rail CSV file
output_folder = '../data/output_images/LosAngelesMR/'
output_csv = output_folder+'LAA.csv'

# Set folder for new images
image_folder = output_folder+'set_3/'

### Import Routes

In [16]:
routes = gpd.read_file(rail_file)
routes = routes.to_crs({'init' :'epsg:4269'})
routes.head()

,Id,OBJECTID,SHAPE_Leng,layer,path,geometry
0,0.0,NaN,NaN,801_Track_0316,/Users/jessedecker/projects/rail_classificatio...,LINESTRING (-118.1893707098661 33.777799051058...
1,0.0,NaN,NaN,801_Track_0316,/Users/jessedecker/projects/rail_classificatio...,LINESTRING (-118.1937438467048 33.776128475602...
2,0.0,NaN,NaN,801_Track_0316,/Users/jessedecker/projects/rail_classificatio...,LINESTRING (-118.1937438467048 33.776128475602...
3,0.0,NaN,NaN,801_Track_0316,/Users/jessedecker/projects/rail_classificatio...,LINESTRING (-118.1937425194715 33.768454143462...
4,0.0,NaN,NaN,801_Track_0316,/Users/jessedecker/projects/rail_classificatio...,LINESTRING (-118.1929738864244 33.767995720331...


In [5]:
routes.shape

(84, 5)

In [17]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

9515 points


In [54]:
'''
Get points for 1 or more routes
'''

rows = [1]

points = []

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')

9792 points


##### Get nth number

In [18]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
curr_df.head()

,Name,Longitude,Latitude,Catenary
0,-117.91040193411446_34.13467863417091,-117.910402,34.134679,0
1,-117.92753120066565_34.12962536413221,-117.927531,34.129625,0
2,-117.96668931179072_34.132550459539395,-117.966689,34.132550,0
3,-117.99040536288051_34.13243703485112,-117.990405,34.132437,1
4,-117.99120183319282_34.13449350348269,-117.991202,34.134493,1


In [21]:
'''
Get subselection of points excuding current points
'''
rows = curr_df.index.tolist()
curr_points = []
for row in rows:
    tmp = curr_df.iloc[row].Name
    space = tmp.find('_') 
    longitude = tmp[:space]
    latitude = tmp[space+1:]
    curr_points.append((longitude,latitude))

route_points = []
for point in range(0,len(points),95):
    tmp = points[point]
    if tmp not in curr_points:
        route_points.append(tmp)

print(len(route_points))

101


In [22]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,-118.18937070986607_33.777799051058786,-118.189371,33.777799
1,-118.243363765737_33.99830163525223,-118.243364,33.998302
2,-118.24320991534833_33.96775139460449,-118.243210,33.967751
3,-118.22500642637812_33.903869763483534,-118.225006,33.903870
4,-118.1917648035579_33.81121925785609,-118.191765,33.811219


In [23]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [24]:
'''
Get satellite preview for image
'''

row = 1

figure_layout = {
    'width': '100%',
    'height': '800px'
}
# fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig = gmaps.figure(center = (list(routes[routes['sub_type']=='1'].iloc[5].geometry.coords)[0][1],
                            list(routes[routes['sub_type']=='1'].iloc[5].geometry.coords)[0][0]), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)

fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [24]:
'''
Export points
'''
# Write new
# df.to_csv(output_csv,header=True,index=None)


# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

In [25]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Collection.ipynb'))
img_folder = root[:-9]+image_folder[3:]
img_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = img_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 
    else:
        print(r.status_code)